
# Arctic Hillslope Failures Analysis
## NDVI-Based Research Expedition for Identifying Potential Slumps

### Introduction
This notebook documents the analysis performed on Sentinel-2 imagery data from site `s001` to identify areas prone to hillslope failures, also known as slumps. The analysis primarily focuses on NDVI (Normalized Difference Vegetation Index) calculations to monitor vegetation health over time, detecting significant changes that may indicate environmental stress.


### 1. Data Loading and Preparation

In [2]:

import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Path to the directory containing Sentinel data
data_dir = '/Users/joshuapiesner/Documents/GitHub/Math76_FinalProject/Data/Sentinel-s001'  # Path to sentinal s001 data

# List all the Sentinel-2 .tif files
tif_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.tif')])

# Initialize a dictionary to store NDVI results by date
ndvi_by_date = {}

# Loop over each .tif file, calculate NDVI, and store the results
for tif_file in tif_files:
    tif_path = os.path.join(data_dir, tif_file)
    with rasterio.open(tif_path) as src:
        raster_data = src.read()
        red_band = raster_data[2]  # Band 3 is typically Red
        nir_band = raster_data[3]  # Band 4 is typically NIR
        ndvi = (nir_band.astype(float) - red_band.astype(float)) / (nir_band + red_band)
        date = tif_file.split('_')[1].split('.')[0]  # Extract date from filename
        ndvi_by_date[date] = ndvi

# Display metadata from one file
with rasterio.open(os.path.join(data_dir, tif_files[0])) as src:
    raster_meta = src.meta
raster_meta


FileNotFoundError: [Errno 2] No such file or directory: '/Users/joshuapiesner/Documents/GitHub/Math76_FinalProject_data/Data/Sentinel-s001'

### 2. NDVI Calculation and Visualization

In [ ]:

# Display all NDVI maps over time
fig, axs = plt.subplots(nrows=len(ndvi_by_date)//3 + 1, ncols=3, figsize=(18, 18))
fig.suptitle('NDVI Over Time for Site s001', fontsize=16)

for ax, (date, ndvi_map) in zip(axs.flatten(), ndvi_by_date.items()):
    im = ax.imshow(ndvi_map, cmap='RdYlGn', vmin=-1, vmax=1)
    ax.set_title(date)
    ax.axis('off')

fig.colorbar(im, ax=axs, fraction=0.02, pad=0.04)
plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()


### 3. Temporal NDVI Analysis

In [ ]:

# Calculate the mean NDVI for each date
mean_ndvi = {date: np.nanmean(ndvi_map) for date, ndvi_map in ndvi_by_date.items()}

# Convert the results to a DataFrame
ndvi_df = pd.DataFrame(list(mean_ndvi.items()), columns=['Date', 'Mean_NDVI'])
ndvi_df['Date'] = pd.to_datetime(ndvi_df['Date'])

# Sort the DataFrame by date
ndvi_df = ndvi_df.sort_values('Date')

# Plotting the NDVI time series
plt.figure(figsize=(12, 6))
plt.plot(ndvi_df['Date'], ndvi_df['Mean_NDVI'], marker='o', linestyle='-', color='green')
plt.title('Mean NDVI Over Time for Site s001')
plt.xlabel('Date')
plt.ylabel('Mean NDVI')
plt.grid(True)
plt.show()

ndvi_df


### 4. NDVI Change Detection

In [ ]:

# Calculate the difference in NDVI between the earliest and latest dates
earliest_date = min(ndvi_by_date.keys())
latest_date = max(ndvi_by_date.keys())

ndvi_change = ndvi_by_date[latest_date] - ndvi_by_date[earliest_date]

# Plot the NDVI change map
plt.figure(figsize=(10, 8))
plt.imshow(ndvi_change, cmap='RdYlBu', vmin=-1, vmax=1)
plt.colorbar(label='NDVI Change (Latest - Earliest)')
plt.title('NDVI Change Between {} and {}'.format(earliest_date, latest_date))
plt.axis('off')
plt.show()


### 5. High-Risk Zones Identification

In [ ]:

# Define a threshold for significant NDVI decline
threshold = -0.2

# Create a binary mask for significant NDVI decreases
high_risk_zones = ndvi_change < threshold

# Plot the high-risk zones on the NDVI change map
plt.figure(figsize=(10, 8))
plt.imshow(ndvi_change, cmap='RdYlBu', vmin=-1, vmax=1)
plt.imshow(high_risk_zones, cmap='Reds', alpha=0.5)
plt.colorbar(label='NDVI Change (Latest - Earliest)')
plt.title('High-Risk Zones for Slumps (NDVI Decrease > 0.2)')
plt.axis('off')
plt.show()

# Calculate the area of high-risk zones
high_risk_area = np.sum(high_risk_zones) * (raster_meta['transform'][0] ** 2)  # Area in square meters
high_risk_area


### 6. Conclusion and Recommendations


The analysis identifies areas with significant vegetation loss over time, suggesting potential hillslope failures or slumping. These high-risk zones should be monitored closely, and additional data such as a Digital Elevation Model (DEM) could further refine the analysis.
